# Link to extracted features: https://www.dropbox.com/scl/fo/1r24ouz7byw39gwubmc9i/AJC7aJRWk7IUZgq9rsyz1Bg?rlkey=vcr9luzjjylucdgrt0tm11ut6&st=6grz4iw7&dl=0

In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoModel, ViTFeatureExtractor, ViTModel
from PIL import Image
import pickle

In [3]:
ds1 = torch.load("dataset\\part_one_dataset\\train_data\\1_train_data.tar.pth");
print(ds1.keys())
data, targets = ds1['data'], ds1['targets']

dict_keys(['data', 'targets'])


C:\Users\rajud\AppData\Local\Temp\ipykernel_1900\581164180.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ds1 = torch.load("dataset\\part_one_dataset\\train_data\\1_tra

In [4]:
ds1 = torch.load("dataset\\part_one_dataset\\eval_data\\1_eval_data.tar.pth");
print(ds1.keys())
data_eval, targets_eval = ds1['data'], ds1['targets']

dict_keys(['data', 'targets'])


C:\Users\rajud\AppData\Local\Temp\ipykernel_1900\1793198265.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ds1 = torch.load("dataset\\part_one_dataset\\eval_data\\1_eva

In [5]:
def resize(image, size = (224,224)):
    pil_image = Image.fromarray(image)
    resized_pil_image = pil_image.resize(size, Image.Resampling.BICUBIC)
    resized_image = np.array(resized_pil_image)
    return resized_image

In [7]:
for pos in range(6,7):    
    model_name = "google/vit-base-patch16-224"
    feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)
    model = ViTModel.from_pretrained(model_name)

    ds = torch.load(f"dataset\\part_one_dataset\\train_data\\{pos}_train_data.tar.pth");
    print(ds.keys())
    data = ds['data']

    ts = torch.load(f"dataset\\part_one_dataset\\eval_data\\{pos}_eval_data.tar.pth");
    print(ts.keys())
    data_eval, targets_eval = ts['data'], ts['targets']

    train_x = []
    for i in range(len(data)):
        inputs = feature_extractor(images=resize(data[i]), return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inputs)
        features = outputs.last_hidden_state[:, 0, :]
        train_x.append(features[0])

    test_x = []
    for i in range(len(data_eval)):
        inputs = feature_extractor(images=resize(data_eval[i]), return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inputs)
        features = outputs.last_hidden_state[:, 0, :]
        test_x.append(features[0])
    
    np.savez(f'ds{pos}_train_new.npz', x=train_x)
    np.savez(f'ds{pos}_eval_new.npz', x=test_x, y=targets_eval)

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\rajud\AppData\Local\Temp\ipykernel_1900\1040828521.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serializ

dict_keys(['data'])
dict_keys(['data', 'targets'])


In [26]:
# import numpy as np
# import matplotlib.pyplot as plt
# for i in range(2):    
#     plt.imshow(data[i])
#     plt.axis('off')  # Hide axes for a cleaner view
#     plt.show()

In [23]:
def apply_conv(image, kernel, stride=(16,16)):
    img_h, img_w, img_c = image.shape
    kernel_h, kernel_w, kernel_c = kernel.shape
    stride_h, stride_w = stride

    assert img_c == kernel_c, "Kernel depth must match the image depth."
    out_h = (img_h - kernel_h) // stride_h + 1
    out_w = (img_w - kernel_w) // stride_w + 1
    output = np.zeros((out_h, out_w))
    for i in range(out_h):
        for j in range(out_w):
            start_h, start_w = i * stride_h, j * stride_w
            patch = image[start_h:start_h + kernel_h, start_w:start_w + kernel_w, :]

            output[i, j] = np.sum(patch * kernel)
    return output